Ajouter un fichier composition pour noter les échantillons utilisés

In [1]:
import pandas as pd
import random as rd
import datetime as dt
import pickle,glob,re,yaml

In [2]:
cumule=True
#repNom="/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/"
repNom="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Samples/Verbes3/"
repNom="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Samples/Verbes3Rnd/"
#repNomSortie="/Users/gilles/Box Sync/2015-Data/FlexionAdjectifs/"
repNomSortie="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Longitudinales/"
repNomSortie="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/LongitudinalesRnd/"

In [3]:
listeTirages=glob.glob(repNom+"*-Tirage.pkl")

In [4]:
def tiragesNom(nom):
    result=[tirage for tirage in listeTirages if nom in tirage]
    return result

In [5]:
compositionLongitudinale=[("1Ko",10),("10Ko",9),("100Ko",9),("1Mo",19)]
compositionLongitudinale=[("10Mo",20)]
compositionLongitudinale=[("10Ko",10),("100Ko",9),("1Mo",9)]
compositionLongitudinale=[("1000Ko",10)]

In [6]:
serieLongitudinale=[]
for serie,nbSerie in compositionLongitudinale:
    for nom in rd.sample(tiragesNom(serie),nbSerie):
        m=re.match(ur"^.*-%s-(\d{2})-T.*-F.*-Tirage.*"%serie,nom)
        if m:
#            print m.group(1), nom
            serieLongitudinale.append(nom)
        else:
            print "mauvais nom de fichier",nom
serieLongitudinale

['/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Samples/Verbes3Rnd/MGC-1000Ko-30-T1000000-F29754-Tirage.pkl',
 '/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Samples/Verbes3Rnd/MGC-1000Ko-25-T1000000-F29871-Tirage.pkl',
 '/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Samples/Verbes3Rnd/MGC-1000Ko-40-T1000000-F29822-Tirage.pkl',
 '/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Samples/Verbes3Rnd/MGC-1000Ko-34-T1000000-F29725-Tirage.pkl',
 '/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Samples/Verbes3Rnd/MGC-1000Ko-04-T1000000-F29837-Tirage.pkl',
 '/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Samples/Verbes3Rnd/MGC-1000Ko-19-T1000000-F29883-Tirage.pkl',
 '/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Samples/Verbes3Rnd/MGC-1000Ko-07-T1000000-F29830-Tirage.pkl',
 '/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Samples/Verbes3Rnd/MGC-1000Ko-31-T1000000-F29812-Tirage.pkl',
 '/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Samples/Verbes3Rnd/MGC-1000

In [7]:
def lireLexique(nomLexique):
    with open(nomLexique, 'rb') as input:
        lexique=pickle.load(input)
    return lexique

In [8]:
cumulLexique=lireLexique(serieLongitudinale[0])
cumulLexique["tir1"]=0

In [9]:
timeStamp=dt.datetime.now().strftime("%y%m%d-%H%M")
for nTirage,tirage in enumerate(serieLongitudinale):
    lexique=lireLexique(tirage)
    if cumule:
        cumulLexique["tir1"]=cumulLexique["tir1"]+lexique["tir1"]
    else:
        cumulLexique["tir1"]=lexique["tir1"]
    lexique1=cumulLexique[cumulLexique["tir1"]>0]
    nbTokens=lexique1["tir1"].sum()
    nbTypes=lexique1["tir1"].count()
    print "(%d,%d),"%(nbTokens,nbTypes)
    sortieNom=repNomSortie+"Longitudinal-%02d-T%d-F%d-%s.pkl"%(nTirage,nbTokens,nbTypes,timeStamp)
    with open(sortieNom,"wb") as output:
        pickle.dump(lexique1, output, pickle.HIGHEST_PROTOCOL)

(1000000,29754),
(2000000,37300),
(3000000,41968),
(4000000,45290),
(5000000,47948),
(6000000,50087),
(7000000,51910),
(8000000,53440),
(9000000,54798),
(10000000,56057),


In [10]:
nomComposition=repNomSortie+"Longitudinal-Composition.yaml"
yaml.safe_dump(serieLongitudinale, file(nomComposition, 'w'), encoding='utf-8', allow_unicode=True)